In [3]:
#曲线类  
import math;
class  Curve():  
    def __init__(self, startPoint = [0, 0], startAngle = 0, span = 100, curveRate = 0):  
        #起点  
        self.startPoint = startPoint;  
        #起点的斜率  
        self.startAngle = startAngle;  
        #跨度  
        self.span = span;  
        #曲率，1/曲率半径， 直线为0  
        self.curveRate = curveRate;  
        #路径  
        self.path = [];  
  
    def setStartPoint(self, point):  
        self.startPoint = point;  
  
    #曲线长度，或跨度  
    def setSpan(self, span):  
        self.span = span;  
  
    #设置曲率  
    #曲率的定义：单位曲线长度的两端点的象限角的差，与该长度的比值。  
    def setCurveRate(self, rate = 0):  
        #rate = 0, 为直线  
        #rate = 1 / R, 为曲率半径R的圆弧  
        #rate的概念：R长度上转过 1 个弧度单位。  
        self.curveRate = rate;  
  
    #区格划分  
    def lineSpacing(self):  
        #设置10个长度单位为一个区格，这个值可以自定  
        cell = 10;  
  
        if (self.span <= cell):  
            return [0, self.span];  
        else:  
            n = int(self.span//cell);  
            m = round((self.span-n*cell)/2, 2);  
            if (abs(m) < 1e-6):  
                a = [0];  
            else:  
                a = [0, m];  
  
            for i in range(1, n):  
                a.append(round(m+i*cell, 2));  
  
            a.append(self.span);  
            return a;  
  
    #计算路径，必须调用此方法  
    def calcPath(self):  
        if (self.curveRate != 0):  
            lSpace = self.lineSpacing();  
  
            angle = self.startAngle;  
            dangle = 0;  
            len_ = len(lSpace);  
  
            self.path.append([self.startPoint, angle]);  
  
            for i in range(len_-1):  
                ds = lSpace[i+1] - lSpace[i];  
                dangle = ds*self.curveRate;  
                angle += dangle;  
  
                dx = ds*math.cos(angle);  
                dy = ds*math.sin(angle);  
                p = self.path[-1][0];  
                x, y = round(p[0]+dx, 2), round(p[1]+dy, 2);  
                self.path.append([[x, y], round(angle, 4)]);  
        else:  
            angle = self.startAngle;  
            self.path.append([self.startPoint, angle]);  
            x, y = self.startPoint[0]+ self.span*math.cos(angle), self.startPoint[1]+ self.span*math.sin(angle);  
            self.path.append([[round(x, 2), round(y, 2)], angle]);  
  
    #结果获取  
    def getPath(self):  
        a = [];  
        for i in range(len(self.path)):  
            a.append(self.path[i][0]);  
  
        return a;  
  
    def getAngle(self):  
        a = [];  
        for i in range(len(self.path)):  
            a.append(self.path[i][1]);  
  
        return a;  
  
    def getEndpoint(self):  
        return self.path[-1][0];  
  
    def getEndangle(self):  
        return self.path[-1][1];

In [4]:
def tmp8():  
    a = Curve();  
    a.setStartPoint([0, 0]);  
    a.setSpan(100.5);  
    lSpace = a.lineSpacing();  
    print(lSpace);  
  
    #设置曲线的曲率  
    a.setCurveRate(1/50);  
    a.calcPath();  
    print(a.getPath());  
    print(a.getAngle());
    
tmp8()    

[0, 0.25, 10.25, 20.25, 30.25, 40.25, 50.25, 60.25, 70.25, 80.25, 90.25, 100.5]
[[0, 0], [0.25, 0.0], [10.04, 2.04], [19.23, 5.98], [27.46, 11.67], [34.39, 18.88], [39.75, 27.32], [43.33, 36.66], [44.98, 46.52], [44.64, 56.51], [42.32, 66.24], [37.96, 75.52]]
[0, 0.005, 0.205, 0.405, 0.605, 0.805, 1.005, 1.205, 1.405, 1.605, 1.805, 2.01]


In [5]:
#描绘类  
class Sketch():  
    def __init__(self):  
        self.path = [];  
        self.currentPoint = None;  
        self.currentAngle = None;  
  
    #起始点的角度  
    @property  
    def startAngle(self):  
        return self.currentAngle;  
 
    @startAngle.setter  
    def startAngle(self, angle):  
        self.currentAngle = angle;  
  
    #给起始角度增加一个角度，相当于逆时针旋转  
    #顺时针时输入负值  
    def addAngle(self, angle):  
        #一般输入默认为角度制，需要转化为弧度  
        angle = angle /(180/3.1415926);  
        if self.currentAngle == None:  
            self.currentAngle = angle;  
        else:  
            self.currentAngle += angle;  
      
    #设置起始点的位置  
    def setStartPoint(self, point):  
        self.currentPoint = point;  
  
    #画曲线  
    def curve(self, span = 100, curveRate = 0):  
        if self.currentPoint == None:  
            if (self.currentAngle == None):  
                c = Curve([0, 0], 0, span, curveRate);  
            else:  
                c = Curve([0, 0], self.currentAngle, span, curveRate);  
            c.calcPath();  
            self.path.append(c.getPath());  
        else:  
            c = Curve(self.currentPoint, self.currentAngle, span, curveRate);  
            c.calcPath();  
            self.path.append(c.getPath());  
  
        self.currentPoint = c.getEndpoint();  
        self.currentAngle = c.getEndangle();  
  
    def getPath(self):  
        return self.path;  
  
    #连接各条曲线路径  
    def linkPath(self):  
        if self.path != []:  
            path = [];  
            for i in range(len(self.path)):  
                path_ = self.path[i];  
                for j in range(len(path_)):  
                    path.append(path_[j]);  
  
            return path;  
  
        return self.path; 

In [6]:
def tmp8():  
    ''''' 
    a = Curve(); 
    a.setStartPoint([0, 0]); 
    a.setSpan(100.5); 
    lSpace = a.lineSpacing(); 
    print(lSpace); 
 
    #设置曲线的曲率 
    a.setCurveRate(1/50); 
    a.calcPath(); 
    print(a.getPath()); 
    print(a.getAngle()); 
 
    '''  
  
    a = Sketch();  
    a.curve(200, 0);  
    a.curve(200, 1/50);  
    a.curve(200, -1/50);  
    print(a.linkPath()); 

tmp8()    

[[0, 0], [200.0, 0.0], [200.0, 0.0], [209.8, 1.99], [219.01, 5.88], [227.26, 11.53], [234.23, 18.7], [239.63, 27.11], [243.25, 36.43], [244.95, 46.28], [244.66, 56.28], [242.39, 66.02], [238.23, 75.11], [232.34, 83.19], [224.97, 89.94], [216.4, 95.1], [206.98, 98.45], [197.08, 99.86], [187.1, 99.28], [177.43, 96.72], [168.46, 92.29], [160.55, 86.17], [154.01, 78.6], [154.01, 78.6], [146.1, 72.48], [137.13, 68.05], [127.46, 65.49], [117.48, 64.91], [107.58, 66.32], [98.16, 69.67], [89.59, 74.83], [82.22, 81.58], [76.33, 89.66], [72.17, 98.75], [69.9, 108.49], [69.61, 118.49], [71.31, 128.34], [74.93, 137.66], [80.33, 146.07], [87.3, 153.24], [95.55, 158.89], [104.76, 162.78], [114.56, 164.77], [124.56, 164.77]]
